## Introdução

O objetivo deste trabalho é gerar um classificador knn para a base de dados [adult](http://mlr.cs.umass.edu/ml/datasets/Census+Income). Esta base de dados contém os dados do censo americado de 19xx e 
Para isto o primeiro passo é ler a base de dados e tratá-los para ser possível :
* Ler a base de dados e tratar eventuais inconsistências
* Remove colunas redudantes
* Transformar as categorias em encoding numérico 
* Normalizar atributos numéricos para variarem no intervalo de 0 a 1

### Importando bibliotecas e lendo base de dados

In [60]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import scipy.stats as stats


import numbers

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
CURRENT_DIR = os.path.abspath(os.path.dirname(__name__))
DATA_DIR = os.path.join(CURRENT_DIR, 'data')

TRAIN_DATA_FILE = os.path.join(DATA_DIR, 'adult.data')
TEST_DATA_FILE = os.path.join(DATA_DIR, 'adult.test')

from collections import OrderedDict
#extracted from 
data_types = OrderedDict([
    ("age", "int"),
    ("workclass", "category"),
    ("final_weight", "int"),  # originally it was called fnlwgt
    ("education", "category"),
    ("education_num", "int"),
    ("marital_status", "category"),
    ("occupation", "category"),
    ("relationship", "category"),
    ("race", "category"),
    ("sex", "category"),
    ("capital_gain", "float"),  # required because of NaN values
    ("capital_loss", "int"),
    ("hours_per_week", "int"),
    ("native_country", "category"),
    ("income_class", "category"),
])
target_column = "income_class"

#reading data
def read_dataset(path):
    data = pd.read_csv(
        path,
        names=data_types,
        index_col=None,
        dtype=data_types,
        comment='|',  
        skipinitialspace=True
    )
    #data = data.drop('final_weight', axis=1)
    return data

train_data = read_dataset(TRAIN_DATA_FILE)
test_data = read_dataset(TEST_DATA_FILE)

#concatena teste a data para avaliar o pré processamento
data = pd.concat([test_data, train_data])
(data.describe(include='all'))

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_class
count,48842.000000,48842,4.884200e+04,48842,48842.000000,48842,48842,48842,48842,48842,48842.000000,48842.000000,48842.000000,48842,48842
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,4
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,33906,NaN,15784,NaN,22379,6172,19716,41762,32650,NaN,NaN,NaN,43832,24720
mean,38.643585,NaN,1.896641e+05,NaN,10.078089,NaN,NaN,NaN,NaN,NaN,1079.067626,87.502314,40.422382,NaN,NaN
std,13.710510,NaN,1.056040e+05,NaN,2.570973,NaN,NaN,NaN,NaN,NaN,7452.019058,403.004552,12.391444,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.175505e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.781445e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.376420e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


Nesta primeira importação vemos que a coluna 'final_weight' contem apenas informações sobre a coleta dos dados e não nos diz nada sobre a variável que queremos aprender.

## Pré-processamento
Nesta etapa vamos
* Remover colunas redundante
* Codificar categorias
* Normalizar categorias
* Lidar com dados faltantes

### Remoção de inconsistências
Vemos de cara que algumas classes estão com valores inconsistencias. Por exemplo a classe alvo 'income_class' tem 4 valores quando na verdade deveriam ter apenas 2.


In [61]:
data.income_class.value_counts()

income_class
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

Observando o value_counts vemos que isto se deve a um '.' adicional em uma das categorias.
Deve ser limpado:

In [62]:
data['income_class'] = data.income_class.str.rstrip('.').astype('category')
data.income_class.value_counts()

income_class
<=50K    37155
>50K     11687
Name: count, dtype: int64

Também notamos que algumas categorias numéricas tem valores '9' de forma muito frequente o que pode indicar um placeholder para valores faltantes nestas categorias.
Observando a frequência dos 5 valores mais comuns de 'capital_gain' e 'hours_per_week'

In [63]:
import heapq
hours_per_week_counts = data.hours_per_week.value_counts()
data.hours_per_week.value_counts()[hours_per_week_counts.index.isin(heapq.nlargest(5, data.hours_per_week.unique()))]

hours_per_week
99    137
98     14
96      9
97      2
95      2
Name: count, dtype: int64

In [64]:
import heapq
capital_gain_counts = data.capital_gain.value_counts()
data.capital_gain.value_counts()[capital_gain_counts.index.isin(heapq.nlargest(5, data.capital_gain.unique()))]

capital_gain
99999.0    244
27828.0     58
25236.0     14
34095.0      6
41310.0      3
Name: count, dtype: int64

Como esperado vemos que estes valores estão muito mais frequentes que suas redondezas, indicando que devem ser removidos.
Para isto vamos substituí-los pela média destas colunas já excluindo estes valores.

In [68]:
capital_mean = np.mean(data.capital_gain[data.capital_gain != 99999])
data['capital_gain'] = data['capital_gain'].replace(99999, capital_mean)
hours_per_week_mean = np.mean(data.hours_per_week[data.hours_per_week != 99])
data['hours_per_week'] = data['hours_per_week'].replace(99, hours_per_week_mean)

### Dados faltantes
Observando os dados vemos que as colunas 'workclass', 'occuptation' e 'native_country' tem dados faltantes indicados por '?'.

In [ ]:
(data == '?').sum(axis=0)

age                  0
workclass         2799
final_weight         0
education            0
education_num        0
marital_status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     857
income_class         0
dtype: int64

Nestas categorias vamos substituir os valores faltantes pelos mais frequentes nestas classes.

In [ ]:
data['workclass'] = data['workclass'].replace('?', 'Private')
data['occupation'] = data['occupation'].replace('?', 'Prof-specialty')
data['native_country'] = data['native_country'].replace('?', 'United-States')

### Avaliação de Classes Correlacionadas

Observando as colunas é imediato que algumas classes devem ter grande correlação entre si. Por exemplo: é de se esperar uma correlação grande entre 'relationship' e 'marital_status', 'education' e 'education_num'.
Para garantir que não estamos introduzindo nenhum viés adicional a base de dados vamos avaliar a correlação entre estas colunas.
Para avaliar a correlação entre duas categorias vamos utilizar a métrica 'Cramers V'.

In [ ]:
# le = preprocessing.LabelEncoder()
# marital_status = le.fit_transform(data.marital_status)
# relationship = le.fit_transform(data.relationship)
# stats.pointbiserialr(marital_status, relationship)
def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    n = n.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

confusion_matrix = pd.crosstab(data['marital_status'], data['relationship'])
cramers_v(confusion_matrix=confusion_matrix)

np.float64(0.4880589431633566)

Para comparação vamos ver a correlação entre 'marital_status' e 'occupation'


In [ ]:
confusion_matrix = pd.crosstab(data['marital_status'], data['occupation'])
cramers_v(confusion_matrix=confusion_matrix)

np.float64(0.13056760739929651)

In [ ]:
confusion_matrix = pd.crosstab(data['workclass'], data['occupation'])
cramers_v(confusion_matrix)

np.float64(0.3997244048209072)

Para avaliar a correlação entre 'education' e 'education_num' vamos utilizar a correlação biserial e utilizar uma serialização por LabelEncoder para  'education' apenas para verificar a correlação.

In [ ]:
le = preprocessing.LabelEncoder()
# le = preprocessing.LabelEncoder()
education = le.fit_transform(data.education)
stats.pointbiserialr(education, data.education_num)

SignificanceResult(statistic=np.float64(0.3596676843392162), pvalue=np.float64(0.0))

Desta forma, vemos que há uma alta covariância entre as colunas:
* education x education_num
* marital_status x relationship
* ocupation x workclass

Para simplificar o modelo vamos escolher as colunas education_num (por já ser numérica e carregar a informação de "mais anos estudados"), relationship  e occupation (por conterém menos classes) .

### Classes com elementos superrepresentados

Vemos que na coluna 'native_country' 90% dos respondentes tem nacionalidade 'United-States' e além disso, temos 42 categorias diferentes para esta coluna.
Por conta disso vamos trocar essa classe por: native-american com 1 indicando que é americano, 0 indicando que não.

In [ ]:
def usa_map(i):
    if i == 39:
        return 1
    return 0
    
data['native_country'] = data['native_country'].astype('category')
data['native_usa'] = data['native_country'].cat.codes.map(usa_map)
data = data.drop('native_country', axis=1)


Com isto podemos criar uma função para limpar todos os dados e mapear os valores inteiros entre 0 a 1 dividindo pelo máximo da coluna e utilizar HotEncoding para os valores categócios

In [69]:
data.describe(include='all')

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_class
count,48842.000000,48842,4.884200e+04,48842,48842.000000,48842,48842,48842,48842,48842,48842.000000,48842.000000,48842.000000,48842,48842
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,33906,NaN,15784,NaN,22379,6172,19716,41762,32650,NaN,NaN,NaN,43832,37155
mean,38.643585,NaN,1.896641e+05,NaN,10.078089,NaN,NaN,NaN,NaN,NaN,582.412136,87.502314,40.257612,NaN,NaN
std,13.710510,NaN,1.056040e+05,NaN,2.570973,NaN,NaN,NaN,NaN,NaN,2530.307226,403.004552,11.995659,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.175505e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.781445e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.376420e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


In [89]:
def clean_dataset(data):
    data = data.drop('final_weight', axis=1) # drops final_weight
    data = data.drop('workclass', axis=1) # drops workclass
    data = data.drop('education', axis=1) # drops education
    data = data.drop('relationship', axis=1) #drops  relationship

    data['income_class'] = data.income_class.str.rstrip('.').astype('category')

    capital_mean = np.mean(data.capital_gain[data.capital_gain != 99999])
    data['capital_gain'] = data['capital_gain'].replace(99999, capital_mean)
    hours_per_week_mean = np.mean(data.hours_per_week[data.hours_per_week != 99])
    data['hours_per_week'] = data['hours_per_week'].replace(99, hours_per_week_mean)

    #data['workclass'] = data['workclass'].replace('?', 'Private')
    data['occupation'] = data['occupation'].replace('?', 'Prof-specialty')

    # condensa classe native_country
    data['native_country'] = data['native_country'].replace('?', 'United-States')
    data['native_country'] = data['native_country'].astype('category')
    mode = data['native_country'].cat.codes.mode()
    print(mode[0])
    usa_map = lambda a : True if a == mode[0] else False

    native_usa = data['native_country'].cat.codes.map(usa_map)
    data = data.drop('native_country', axis=1)
    data = pd.concat([data, native_usa], axis=1)

    #normaliza valores numéricos
    data['age'] = data['age']/90
    data['education_num'] = data['education_num']/16
    data['capital_gain'] = data['capital_gain']/41310.0
    data['capital_loss'] = data['capital_loss']/4356.0
    data['hours_per_week'] = data['hours_per_week']/98

    # one hot enconding 
    marital_oh = pd.get_dummies(data['marital_status'])
    data = data.drop('marital_status', axis=1)
    data = pd.concat([data, marital_oh]).reset_index(drop=True)

    occupation_oh = pd.get_dummies(data['occupation'])
    data = data.drop('occupation', axis=1)
    data = pd.concat([data, occupation_oh]).reset_index(drop=True)

    race_oh = pd.get_dummies(data['race'])
    data = data.drop('race', axis=1)
    data = pd.concat([data, race_oh]).reset_index(drop=True)

    sex_oh = pd.get_dummies(data['sex'])
    data = data.drop('sex',axis=1)
    data = pd.concat([data, sex_oh]).reset_index(drop=True)
    #drop duplicates 
    data = data.drop_duplicates()

    #saída 
    y = data['income_class']
    data = data.drop('income_class', axis=1)
    return data, y


In [90]:
clean_test, clean_output = clean_dataset(test_data)
clean_test.describe(include='all')

C:\Users\MateusVendramini\AppData\Local\Temp\ipykernel_10264\3120388230.py:15: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data['occupation'] = data['occupation'].replace('?', 'Prof-specialty')
C:\Users\MateusVendramini\AppData\Local\Temp\ipykernel_10264\3120388230.py:18: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data['native_country'] = data['native_country'].replace('?', 'United-States')


37


,age,education_num,capital_gain,capital_loss,hours_per_week,0,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,...,Sales,Tech-support,Transport-moving,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,White,Female,Male
count,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000,8041,7,7,7,7,...,15,15,15,6,6,6,6,6,3,3
unique,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,NaN,NaN,NaN,NaN,NaN,6997,6,6,6,6,...,14,14,14,5,5,5,5,5,2,2
mean,0.468770,0.635516,0.026702,0.040003,0.415763,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.162771,0.189096,0.082167,0.127262,0.152409,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.188889,0.062500,0.000000,0.000000,0.010204,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.344444,0.562500,0.000000,0.000000,0.357143,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.455556,0.625000,0.000000,0.000000,0.408163,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.577778,0.812500,0.000000,0.000000,0.510204,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clean_train, train_output = clean_dataset(train_data)